In [0]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#with open('/content/drive/My Drive/Colab Notebooks/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/Colab\ Notebooks/foo.txt
#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

In [0]:
import sys
sys.path
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import time
import os
import re
import csv
DEBUG = 1
from datetime import datetime, timedelta
#import dateparser
import calendar

In [0]:
def extractQuestionTitle(driver):
    try:
        question_title = driver.find_element_by_xpath('.//span[contains(@class, "ui_qtext_rendered_qtext")]').text
    except Exception as e:
        question_title = "error"
    return question_title

In [0]:
def extractTopics(driver):
    topic_list = []
    
    try:
        topic_area = driver.find_element_by_xpath(".//div[contains(@class,'QuestionTopicHorizontalList')]")
    except:
        topic_list = ['error']
        return topic_list
    
    topic_list = topic_area.text.split('\n')
           
    return topic_list

In [0]:
def scrollDownPage(driver, speed):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [0]:
def extractRespondContent(driver):
    try:
        respond_content = driver.find_element_by_xpath(".//div[@class = 'q-text']").text
    except Exception as e:
        respond_content = "error"
        
    return respond_content

In [0]:
def extractRespondViewAndUpvote(driver):
    try:
        respond_stat = driver.find_element_by_xpath(".//div[@class = 'q-text qu-mt--small qu-color--gray_light qu-fontSize--small qu-passColorToLinks']").text
        
        view_text = respond_stat.split(" view")[0]
        if "k" in view_text:
            respond_view = int(float((view_text.split("k"))[0]) * 1000)
        elif "m" in view_text:
            respond_view = int(float((view_text.split("m"))[0]) * 1000000)
        else:
            respond_view = int(view_text)
        
        respond_upvote = 0
        if "Upvoter" in respond_stat:
            upvote_text = (respond_stat.split("View ")[1]).split(" ")[0]
            if "k" in upvote_text:
                respond_upvote = int(float((upvote_text.split("k"))[0]) * 1000)
            elif "m" in upvote_text:
                respond_upvote = int(float((upvote_text.split("m"))[0]) * 1000000)
            else:
                respond_upvote = int(upvote_text)
        else:
            respond_upvote = 0
            
    except:
        respond_view = 0
        respond_upvote = 0
        
    return respond_view, respond_upvote

In [0]:
def extractDate(driver):
    try:
        log = driver.find_element_by_xpath(".//p[@class = 'log_action_bar']")
        date = log.text.split("· ")[1]
    except:
        date = "error"
    return date

In [0]:
def extractAuthor(driver):
    block_header = driver.find_element_by_xpath (".//div[@class = 'feed_item_activity']")
    try:
        user_name = block_header.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except Exception:
        try:
            user_name = block_header.find_element_by_xpath(".//a[@class = 'user']").text
        except Exception as e:
            user_name = "error"
    
    return user_name

In [0]:
def log_stat_extraction(driver, driver2):
    # return element
    respond_author_list = []
    respond_content_list = []
    respond_upvote_list = []
    respond_view_list = []
    respond_date_list = []
    respond_count = 0
    question_author = ''
    question_create_time = ''
    
    try:
        block_list = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")
    except:
        respond_author_list =  ["error"]
        respond_content_list = ["error"]
        respond_upvote_list = ["error"]
        respond_view_list = ["error"]
        respond_date_list = ["error"]
        respond_count = 0
        question_auther = "error"
        question_create_time = "error"
        
        return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_auther, question_create_time
        
    for block in block_list:
        block_error = []
        
        block_header = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']").text
        
        try:
            block_revision = block.find_element_by_xpath (".//div[@class = 'revision']").text
        except:
            continue
            
        if 'answer has since been deleted' in block_revision:
            continue
        elif 'deleted' in block_header:
            continue
        
        if 'Answer' in block_header:
            # click the answer link
            try:
                url2 = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']/a").get_attribute('href')
                driver2.get(str(url2))
                respond_count += 1
            except Exception as e:
                continue
                
            # extract author name(log page and driver 1)
            respond_author = extractAuthor(block)
            respond_author_list.append(respond_author)
            
            # extract content
            respond_content = extractRespondContent(driver2)
            respond_content_list.append(respond_content)
                
            # extract views and upvote
            respond_view, respond_upvote = extractRespondViewAndUpvote(driver2)
            respond_view_list.append(respond_view)
            respond_upvote_list.append(respond_upvote)
                
            # extract date(log page and driver 1)
            respond_date = extractDate(block)
            respond_date_list.append(respond_date)
            
        elif 'Question added by' in block_header:
            # extract author name
            question_author = extractAuthor(block)
            
            # extract date
            question_create_time = extractDate(block)

    return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_author, question_create_time



In [0]:
def extractNoOfFollowerAndView(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_follower = int((stats[0].text.split(' '))[0])
        no_of_view = (stats[1].text.split(' '))[0]
    except:
        no_of_follower = 0
        no_of_view = 0
        
    return no_of_follower, no_of_view

In [0]:
def extractQuestion(driver, driver2, url):
    data = {}
    driver.get(url+'/log')
    data['question_title'] = extractQuestionTitle(driver)
    data['topic_list'] = extractTopics(driver)
    
    scrollDownPage(driver, speed = 5)
    data['name_list'], data['ans_list'], data['upvote_list'], data['view_list'], data['date_list'], data['respond_collected'], data['question_author'], data['create_time'] = log_stat_extraction(driver, driver2)
    
    data['no_of_follower'], data['no_of_view'] = extractNoOfFollowerAndView(driver)
    return data

In [0]:
def readTopics():
    with open('/content/drive/My Drive/Colab Notebooks/topics.txt') as topics:
        topic_array = topics.readlines()
    topic_array = [topic.strip("\n") for topic in topic_array] 
    print('Topics: ')
    print(topic_array)
    print()
    return topic_array    

In [0]:
def readURLs(file):
    with open(file) as URLs:
        URL_array = URLs.readlines()
    URL_array = [URL.strip("\n") for URL in URL_array]
    return URL_array

In [0]:
def connectChrome():
    # instantiate a chrome options object so you can set the size and headless preference
    options = webdriver.ChromeOptions()
    options.add_argument("--headless") # 浏览器不提供可视化页面
    options.add_argument("log-level=3") # suppress console error/warning/info messages when executing
    options.add_argument("--incognito") # 隐身模式启动
    options.add_argument('--no-sandbox') # 以最高权限运行
    options.add_argument('--disable-dev-shm-usage') # overcome limited resource problems
    options.add_argument('blink-settings=imagesEnabled=false')  #不加載圖片, 提升速度
    #driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
    #driver.execute_script("window.scrollTo(0, 0);")
    
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
    # and put it in the current directory
    #chromedriver = "/home/youcef/Documents/chromedriver"
    #os.environ["webdriver.chrome.driver"] = chromedriver     
    browser = webdriver.Chrome('chromedriver', options=options) 
    #browser.get("https://www.quora.com/")
    #time.sleep(2)
    return browser

In [0]:
def crawlURLByTopic(topic):
    file_location = "/content/drive/My Drive/Colab Notebooks/topics/" + topic + "_question_urls.txt"
    try:
        with open(file_location) as f:
            print("URLs of the topic \"" + topic + "\" has been crawled")
    except:
        print("Start crawling the URLs of the topic \"" + topic + "\"")
        URLCount = 0
        driver = connectChrome()
        QuoraURL = "https://www.quora.com/topic/" + topic + "/all_questions"
        print(QuoraURL)
        
        try:
            driver.get(QuoraURL)
        except Exception as e0:
            print("exception e0")
            print("Error on line {}".format(sys.exc_info()[-1].tb_lineno), type(e0).__name__, e0)
            return False
        
        # define pause time for browser
        SCROLL_PAUSE_TIME = 2
        
        # get browser source
        html_source = driver.page_source
        question_count_soup = BeautifulSoup(html_source, "html.parser")

        #  get total number of questions
        question_count_str = question_count_soup.find("a", attrs = {"class": "TopicQuestionsStatsRow"})
        if str(question_count_str) == "None":
            print("Topic does not exist in Quora, so we will skip it.")
            return False
        question_count = question_count_str.contents[0].text 
        if "k" in str(question_count):
            question_count = str(question_count).replace("k", "")
            question_count = int(float(question_count) * 1000)
        if "m" in str(question_count):
            question_count = str(question_count).replace("m", "")
            question_count = int(float(question_count) * 1000000)
        print("Number of questions for this topic : "+ str(question_count))
        if str(question_count) == "0":            
            print("Topic has 0 question, so we will skip it.")
            return False
        
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        question_set = set()
        # infinite while loop, break it when you reach the end of the page or not able to scroll further.
        if int(question_count) > 10:
            start_time_sd = time.time()
            max_time = int(question_count) * 0.25
            if int(question_count) > 8000:
                max_time = 1800
            while True:
                i = 0
                # try to scroll 5 times in case of slow connection
                while True:
                    # Scroll down to one page length
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load page
                    time.sleep(SCROLL_PAUSE_TIME)

                    # get page height in pixels
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    # break this loop when you are able to scroll futher
                    if new_height != last_height:
                        break
                    if i == 1:
                        SCROLL_PAUSE_TIME = 5   
                    if i == 2:
                        if int(question_count) > 500 and int(question_count) < 2000:
                            SCROLL_PAUSE_TIME = 8
                        elif int(question_count) > 2000:
                            SCROLL_PAUSE_TIME = 16    
                        elif int(question_count) < 500:
                            break
                    if i == 3:
                        break
                    i += 1
                SCROLL_PAUSE_TIME = 2    
                # not able to scroll further, break the infinite loop
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                total_time = time.time() - start_time_sd
                if total_time > max_time:
                    print("Max time exceeded")
                    break
        print('getting ressources ...')
        # get html page source
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")
        # question_link is the class for questions
        question_link = soup.find_all("a", attrs = {"class": "question_link"}, href = True)

        # add questions to a set for uniqueness
        for ques in question_link:
            question_set.add(ques)
            
        # write content of set to a file called question_urls.txt
        with open(file_location, 'w') as topic_urls:
            writer = csv.writer(topic_urls)
            for ques in question_set:
                link_url = "http://www.quora.com" + ques.attrs['href']
                writer.writerows([[link_url]])
        
    return True
            

In [0]:
def crawlQuestionByTopic(topic):
    start_time = time.time()
    topic_url_file = "/content/drive/My Drive/Colab Notebooks/topics/" + topic + "_question_urls.txt"
    print(topic_url_file)
    # Open question url file
    file_question_urls = open(topic_url_file, mode = 'r')
    answerFile = "/content/drive/My Drive/Colab Notebooks/" + topic + "_answers.txt"

    #file_users = open(os.path.join(sys.path[0]+"users.txt", mode = 'a'))
    current_lines = file_question_urls.readlines()
    URLs = readURLs(topic_url_file)
    driver = connectChrome()
    driver2 = connectChrome()
    topic_array = {}
    topic_array["topic"] = topic
    topic_array["no_of_follows"] = 0
    questions = {}
    
    print("no. of URL: ", len(URLs))
    print()
    i = 1
    for URL in URLs:
        
        if 'unanswered' in URL:
            print(i, ': ', URL)
            #print('skip unanswered question')
            print()
            i += 1
            continue
            
        print(i, ': ', URL)
        i += 1
        question_array = extractQuestion(driver, driver2, URL)
        questions[str(len(questions))] = question_array
        if not i%100:
            print(question_array)
        print()
        topic_array["questions"] = questions
        crawledData = json.dumps(topic_array)
        with open(answerFile, "w") as file_answers:
            file_answers.write(crawledData)
            file_answers.close()
    return topic

In [0]:
def main():
    topics = readTopics()
    for topic in topics:
        crawled = crawlURLByTopic(topic)
        if crawled:
            crawlQuestionByTopic(topic)
    
if __name__ == "__main__": main()    